In [25]:
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
import uuid

In [31]:
conn = psycopg2.connect(
    host='ud-adp-rolling.cbyis8su6n0i.us-east-2.rds.amazonaws.com',
    user='postgres',
    password='liammurphybbm2',
    port=5432
)

cursor = conn.cursor(cursor_factory=RealDictCursor)

# BBM1 (2020)

### Raw Data -> ADP Data

In [44]:
bbm1_data = pd.read_excel(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBMData.xlsx')
print(bbm1_data.columns)
bbm1_data.head(5)

Index(['team', 'player', 'drafted_round', 'roster_points', 'pick_points',
       'draft_time', 'playoff_round', 'made_playoffs'],
      dtype='object')


,team,player,drafted_round,roster_points,pick_points,draft_time,playoff_round,made_playoffs
0,081ea432-dcf5-4370-932d-b53ac2296900,A.J. Brown,4,109.94,0.0,2020-12-22 17:04:23 UTC,Round 4,1
1,081ea432-dcf5-4370-932d-b53ac2296900,Antonio Brown,15,109.94,11.5,2020-12-22 17:04:23 UTC,Round 4,1
2,081ea432-dcf5-4370-932d-b53ac2296900,Baker Mayfield,13,109.94,0.0,2020-12-22 17:04:23 UTC,Round 4,1
3,081ea432-dcf5-4370-932d-b53ac2296900,Chase Edmonds,10,109.94,0.0,2020-12-22 17:04:23 UTC,Round 4,1
4,081ea432-dcf5-4370-932d-b53ac2296900,D.J. Chark,5,109.94,14.2,2020-12-22 17:04:23 UTC,Round 4,1


# BBM2 (2021)

### Raw Data -> ADP Data

In [ ]:
bbm2_data = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBM_II_Data_Dump_Regular_Season_01312022.csv', delimiter=',')
print(bbm2_data.columns)
bbm2_data.head(5)

Index(['draft_id', 'draft_time', 'clock', 'tournament_entry_id',
       'tournament_round_number', 'player_name', 'position_name', 'bye_week',
       'projection_adp', 'pick_order', 'overall_pick_number',
       'team_pick_number', 'pick_points', 'roster_points', 'playoff_team'],
      dtype='object')


,draft_id,draft_time,clock,tournament_entry_id,tournament_round_number,player_name,position_name,bye_week,projection_adp,pick_order,overall_pick_number,team_pick_number,pick_points,roster_points,playoff_team
0,00009a0e-b1a6-42e0-b1cf-3f884cc0999b,2021-08-28 17:06:42.031664+00:00,30,e1fdb4f8-4da3-4a09-8e6c-ef294c053251,1,Jalen Reagor,WR,14,139.48,1,144,12,31.2,1613.92,0
1,00009a0e-b1a6-42e0-b1cf-3f884cc0999b,2021-08-28 17:06:42.031664+00:00,30,b1e60f0c-2982-4e5c-8f80-da9a104cb290,1,Marquez Callaway,WR,6,107.46,6,115,10,81.9,1525.06,0
2,00009a0e-b1a6-42e0-b1cf-3f884cc0999b,2021-08-28 17:06:42.031664+00:00,30,b1e60f0c-2982-4e5c-8f80-da9a104cb290,1,Amari Cooper,WR,7,31.75,6,30,3,116.3,1525.06,0
3,00009a0e-b1a6-42e0-b1cf-3f884cc0999b,2021-08-28 17:06:42.031664+00:00,30,5388d065-c260-45cc-b9bf-fcb37f767d2c,1,Rhamondre Stevenson,RB,14,171.55,11,158,14,66.5,1669.72,1
4,00009a0e-b1a6-42e0-b1cf-3f884cc0999b,2021-08-28 17:06:42.031664+00:00,30,5388d065-c260-45cc-b9bf-fcb37f767d2c,1,Darrynton Evans,RB,13,201.35,11,203,17,2.8,1669.72,1


In [32]:
bbm2_cond = bbm2_data.copy()[['draft_time', 'player_name', 'position_name', 'projection_adp']]
bbm2_cond['draft_time'] = pd.to_datetime(bbm2_cond['draft_time']) + pd.Timedelta(hours=-15)
bbm2_cond['draft_time'] = bbm2_cond['draft_time'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
bbm2_cond['draft_date'] = bbm2_cond['draft_time'].str[:10]
bbm2_cond.head(5)

,draft_time,player_name,position_name,projection_adp,draft_date
0,2021-08-28T02:06:42Z,Jalen Reagor,WR,139.48,2021-08-28
1,2021-08-28T02:06:42Z,Marquez Callaway,WR,107.46,2021-08-28
2,2021-08-28T02:06:42Z,Amari Cooper,WR,31.75,2021-08-28
3,2021-08-28T02:06:42Z,Rhamondre Stevenson,RB,171.55,2021-08-28
4,2021-08-28T02:06:42Z,Darrynton Evans,RB,201.35,2021-08-28


In [38]:
bbm2_date = bbm2_cond[bbm2_cond['draft_date'] == '2021-08-06']
for player_name in bbm2_date['player_name'].unique():
    player_data = bbm2_date[bbm2_date['player_name'] == player_name]
    adp_counts = player_data['projection_adp'].value_counts()
    print(f"\n{player_name}:")
    for adp, count in adp_counts.items():
        print(f"  ADP {adp}: {count} times")


Damien Williams:
  ADP 187.32: 76 times
  ADP 168.3: 6 times
  ADP 163.34: 5 times
  ADP 164.16: 5 times
  ADP 166.75: 3 times
  ADP 163.54: 3 times
  ADP 165.37: 1 times
  ADP 162.41: 1 times
  ADP 163.56: 1 times
  ADP 162.5: 1 times

Dalvin Cook:
  ADP 2.1: 103 times

George Kittle:
  ADP 26.36: 79 times
  ADP 26.64: 13 times
  ADP 26.91: 10 times
  ADP 26.99: 1 times

AJ Dillon:
  ADP 101.68: 77 times
  ADP 97.65: 7 times
  ADP 96.46: 5 times
  ADP 95.06: 5 times
  ADP 98.79: 4 times
  ADP 95.77: 3 times
  ADP 99.28: 1 times
  ADP 94.75: 1 times

Sam Darnold:
  ADP 190.44: 73 times
  ADP 189.26: 5 times
  ADP 188.96: 3 times
  ADP 189.24: 3 times
  ADP 189.27: 2 times
  ADP 188.58: 2 times
  ADP 188.59: 2 times
  ADP 188.72: 2 times
  ADP 189.34: 1 times
  ADP 189.01: 1 times
  ADP 188.56: 1 times

Zach Wilson:
  ADP 180.31: 74 times
  ADP 186.27: 6 times
  ADP 187.21: 4 times
  ADP 186.64: 3 times
  ADP 186.0: 3 times
  ADP 185.98: 3 times
  ADP 186.56: 3 times
  ADP 186.98: 2 ti

In [39]:
dates = sorted(bbm2_cond['draft_date'].unique())
player_names = bbm2_cond['player_name'].unique()

col_names = ['player_id', 'player_name', 'position'] + dates
bbm2_adp = pd.DataFrame(columns=col_names)

player_info = bbm2_cond.groupby('player_name').first()[['position_name']].reset_index()

for player_name in player_names:
    player_data = player_info[player_info['player_name'] == player_name].iloc[0]
    row_data = [None, player_data['player_name'], player_data['position_name']] + [None] * len(dates)
    bbm2_adp.loc[len(bbm2_adp)] = row_data

In [40]:
for date in dates:
    bbm3_on_date = bbm2_cond[bbm2_cond['draft_date'] == date]
    for idx, row in bbm2_adp.iterrows():
        player_data = bbm3_on_date[bbm3_on_date['player_name'] == row['player_name']]
        adp_counts = player_data['projection_adp'].value_counts()
        if len(adp_counts) == 0:
            adp = 216.0
        else:
            adp = adp_counts.index[0]
        bbm2_adp.loc[idx, date] = adp

In [41]:
bbm2_adp.to_csv(r'C:\Users\Jona\Desktop\tuxff\data\adp_data\bbm2_adp.csv')
bbm2_adp.head(5)

,player_id,player_name,position,2021-05-02,2021-05-03,2021-05-04,2021-05-05,2021-05-06,2021-05-07,2021-05-08,...,2021-08-31,2021-09-01,2021-09-02,2021-09-03,2021-09-04,2021-09-05,2021-09-06,2021-09-07,2021-09-08,2021-09-09
0,None,Jalen Reagor,WR,120.0,115.65,119.49,120.49,120.83,122.09,123.07,...,141.23,141.99,142.31,142.25,142.58,143.48,145.19,146.86,147.78,147.78
1,None,Marquez Callaway,WR,214.71,214.85,215.82,215.57,215.44,215.54,215.29,...,99.91,96.87,93.77,91.94,90.82,88.71,86.73,85.33,84.03,82.74
2,None,Amari Cooper,WR,34.0,34.11,35.1,35.09,35.16,35.59,35.28,...,32.12,32.07,32.03,32.13,32.13,32.09,32.24,32.36,32.25,32.11
3,None,Rhamondre Stevenson,RB,245.0,209.19,209.21,209.3,207.79,206.28,204.88,...,155.4,150.54,146.35,142.51,140.26,139.07,138.6,138.42,138.7,139.55
4,None,Darrynton Evans,RB,208.0,208.57,208.92,208.51,209.21,211.0,211.74,...,201.39,201.95,203.11,205.16,208.72,211.93,213.13,213.77,214.0,214.37


### ADP Data -> Postgres DB

In [32]:
bbm2_adp = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\adp_data\bbm2_adp.csv', delimiter=',', index_col=0)
bbm2_adp.head(5)

,player_id,player_name,position,2021-05-02,2021-05-03,2021-05-04,2021-05-05,2021-05-06,2021-05-07,2021-05-08,...,2021-08-31,2021-09-01,2021-09-02,2021-09-03,2021-09-04,2021-09-05,2021-09-06,2021-09-07,2021-09-08,2021-09-09
0,NaN,Jalen Reagor,WR,120.00,115.65,119.49,120.49,120.83,122.09,123.07,...,141.23,141.99,142.31,142.25,142.58,143.48,145.19,146.86,147.78,147.78
1,NaN,Marquez Callaway,WR,214.71,214.85,215.82,215.57,215.44,215.54,215.29,...,99.91,96.87,93.77,91.94,90.82,88.71,86.73,85.33,84.03,82.74
2,NaN,Amari Cooper,WR,34.00,34.11,35.10,35.09,35.16,35.59,35.28,...,32.12,32.07,32.03,32.13,32.13,32.09,32.24,32.36,32.25,32.11
3,NaN,Rhamondre Stevenson,RB,245.00,209.19,209.21,209.30,207.79,206.28,204.88,...,155.40,150.54,146.35,142.51,140.26,139.07,138.60,138.42,138.70,139.55
4,NaN,Darrynton Evans,RB,208.00,208.57,208.92,208.51,209.21,211.00,211.74,...,201.39,201.95,203.11,205.16,208.72,211.93,213.13,213.77,214.00,214.37


In [33]:
bbm2_adp['player_id'] = pd.Series(dtype='object')
for idx, row in bbm2_adp.iterrows():
    cursor.execute("""SELECT player_id 
                   FROM player_info
                   WHERE player_name = %s""",
                   (row['player_name'],))
    
    result = cursor.fetchone()
    
    if result:
        bbm2_adp.loc[idx, 'player_id'] = result['player_id']
    
    else:
        bbm2_adp.loc[idx, 'player_id'] = str(uuid.uuid4())

In [34]:
bbm2_adp.head()

,player_id,player_name,position,2021-05-02,2021-05-03,2021-05-04,2021-05-05,2021-05-06,2021-05-07,2021-05-08,...,2021-08-31,2021-09-01,2021-09-02,2021-09-03,2021-09-04,2021-09-05,2021-09-06,2021-09-07,2021-09-08,2021-09-09
0,16a50953-fbe5-4b26-9170-c3246fe7d5af,Jalen Reagor,WR,120.00,115.65,119.49,120.49,120.83,122.09,123.07,...,141.23,141.99,142.31,142.25,142.58,143.48,145.19,146.86,147.78,147.78
1,8bc11558-7c59-4578-8c40-f27f5f607548,Marquez Callaway,WR,214.71,214.85,215.82,215.57,215.44,215.54,215.29,...,99.91,96.87,93.77,91.94,90.82,88.71,86.73,85.33,84.03,82.74
2,01ed7bc6-254e-4cc1-92f4-f2ba86746625,Amari Cooper,WR,34.00,34.11,35.10,35.09,35.16,35.59,35.28,...,32.12,32.07,32.03,32.13,32.13,32.09,32.24,32.36,32.25,32.11
3,3b5c2580-e73e-4222-8b98-b6dc14690e73,Rhamondre Stevenson,RB,245.00,209.19,209.21,209.30,207.79,206.28,204.88,...,155.40,150.54,146.35,142.51,140.26,139.07,138.60,138.42,138.70,139.55
4,3324bf4a-377b-4bde-8f24-380ab961e382,Darrynton Evans,RB,208.00,208.57,208.92,208.51,209.21,211.00,211.74,...,201.39,201.95,203.11,205.16,208.72,211.93,213.13,213.77,214.00,214.37


In [35]:
bbm2_adp.to_csv(r'C:\Users\Jona\Desktop\tuxff\data\adp_data\bbm2_adp.csv')

In [38]:
bbm2_adp = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\adp_data\bbm2_adp.csv', delimiter=',', index_col=0)
bbm2_adp.head(5)

,player_id,player_name,position,2021-05-02,2021-05-03,2021-05-04,2021-05-05,2021-05-06,2021-05-07,2021-05-08,...,2021-08-31,2021-09-01,2021-09-02,2021-09-03,2021-09-04,2021-09-05,2021-09-06,2021-09-07,2021-09-08,2021-09-09
0,16a50953-fbe5-4b26-9170-c3246fe7d5af,Jalen Reagor,WR,120.00,115.65,119.49,120.49,120.83,122.09,123.07,...,141.23,141.99,142.31,142.25,142.58,143.48,145.19,146.86,147.78,147.78
1,8bc11558-7c59-4578-8c40-f27f5f607548,Marquez Callaway,WR,214.71,214.85,215.82,215.57,215.44,215.54,215.29,...,99.91,96.87,93.77,91.94,90.82,88.71,86.73,85.33,84.03,82.74
2,01ed7bc6-254e-4cc1-92f4-f2ba86746625,Amari Cooper,WR,34.00,34.11,35.10,35.09,35.16,35.59,35.28,...,32.12,32.07,32.03,32.13,32.13,32.09,32.24,32.36,32.25,32.11
3,3b5c2580-e73e-4222-8b98-b6dc14690e73,Rhamondre Stevenson,RB,245.00,209.19,209.21,209.30,207.79,206.28,204.88,...,155.40,150.54,146.35,142.51,140.26,139.07,138.60,138.42,138.70,139.55
4,3324bf4a-377b-4bde-8f24-380ab961e382,Darrynton Evans,RB,208.00,208.57,208.92,208.51,209.21,211.00,211.74,...,201.39,201.95,203.11,205.16,208.72,211.93,213.13,213.77,214.00,214.37


In [39]:
for idx, row in bbm2_adp.iterrows():
    cursor.execute("""
        INSERT INTO player_info (player_id, player_name, position) 
        VALUES (%s, %s, %s)
        ON CONFLICT (player_id) DO NOTHING
    """, (row['player_id'], row['player_name'], row['position']))

conn.commit()

In [40]:
cursor.execute("SELECT player_id, date FROM player_adp")
existing_data = set()
for row in cursor.fetchall():
    date_str = str(row['date'])
    existing_data.add((row['player_id'], date_str))
print('Step 1 Complete')

adp_data = []
for idx, row in bbm2_adp.iterrows():
    for date in bbm2_adp.columns[4:]:
        if (row['player_id'], date) not in existing_data:
            adp_data.append((row['player_id'], date, row[date]))
print(f"Step 2 Complete - {len(adp_data)} new records to insert")

batch_size = 500
for i in range(0, len(adp_data), batch_size):
    batch = adp_data[i:i+batch_size]
    print(f"Starting batch {i//batch_size + 1} with {len(batch)} records...")
    cursor.executemany("""
        INSERT INTO player_adp (player_id, date, adp) 
        VALUES (%s, %s, %s)
    """, batch)
    print(f"Completed batch {i//batch_size + 1}")
    
    conn.commit()
    print(f"Committed batch {i//batch_size + 1}")

Step 1 Complete
Step 2 Complete - 70720 new records to insert
Starting batch 1 with 500 records...
Completed batch 1
Committed batch 1
Starting batch 2 with 500 records...
Completed batch 2
Committed batch 2
Starting batch 3 with 500 records...
Completed batch 3
Committed batch 3
Starting batch 4 with 500 records...
Completed batch 4
Committed batch 4
Starting batch 5 with 500 records...
Completed batch 5
Committed batch 5
Starting batch 6 with 500 records...
Completed batch 6
Committed batch 6
Starting batch 7 with 500 records...
Completed batch 7
Committed batch 7
Starting batch 8 with 500 records...
Completed batch 8
Committed batch 8
Starting batch 9 with 500 records...
Completed batch 9
Committed batch 9
Starting batch 10 with 500 records...
Completed batch 10
Committed batch 10
Starting batch 11 with 500 records...
Completed batch 11
Committed batch 11
Starting batch 12 with 500 records...
Completed batch 12
Committed batch 12
Starting batch 13 with 500 records...
Completed batch

In [41]:
conn.close()
cursor.close()

# BBM3 (2022)

### Raw Data -> ADP Data

In [36]:
BBM3_0 = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBM3\BBM_III_Regular_Season_Dump_Part_00_01302023.csv', delimiter=',')
BBM3_1 = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBM3\BBM_III_Regular_Season_Dump_Part_01_01302023.csv', delimiter=',')
BBM3_2 = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBM3\BBM_III_Regular_Season_Dump_Part_02_01302023.csv', delimiter=',')
BBM3_3 = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBM3\BBM_III_Regular_Season_Dump_Part_03_01302023.csv', delimiter=',')
BBM3_4 = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBM3\BBM_III_Regular_Season_Dump_Part_04_01302023.csv', delimiter=',')
BBM3_5 = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBM3\BBM_III_Regular_Season_Dump_Part_05_01302023.csv', delimiter=',')
BBM3_6 = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBM3\BBM_III_Regular_Season_Dump_Part_06_01302023.csv', delimiter=',')
BBM3_7 = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBM3\BBM_III_Regular_Season_Dump_Part_07_01302023.csv', delimiter=',')
BBM3_8 = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBM3\BBM_III_Regular_Season_Dump_Part_08_01302023.csv', delimiter=',')
BBM3_9 = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBM3\BBM_III_Regular_Season_Dump_Part_09_01302023.csv', delimiter=',')
BBM3_10 = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBM3\BBM_III_Regular_Season_Dump_Part_010_01302023.csv', delimiter=',')
BBM3_11 = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBM3\BBM_III_Regular_Season_Dump_Part_011_01302023.csv', delimiter=',')

In [37]:
BBM3_data = pd.concat([
    BBM3_0, BBM3_1, BBM3_2, BBM3_3, BBM3_4, BBM3_5,
    BBM3_6, BBM3_7, BBM3_8, BBM3_9, BBM3_10, BBM3_11
], ignore_index=True)

BBM3_data.to_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBM3_data.csv', index=False)

In [2]:
bbm3_data = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\BBM3_data.csv', delimiter=',')
print(bbm3_data.columns)
bbm3_data.head(5)

Index(['draft_id', 'draft_time', 'clock', 'draft_entry_id',
       'tournament_entry_id', 'tournament_round_draft_entry_id',
       'tournament_round_number', 'player_name', 'position_name', 'bye_week',
       'projection_adp', 'pick_order', 'overall_pick_number',
       'team_pick_number', 'pick_points', 'roster_points', 'playoff_team'],
      dtype='object')


,draft_id,draft_time,clock,draft_entry_id,tournament_entry_id,tournament_round_draft_entry_id,tournament_round_number,player_name,position_name,bye_week,projection_adp,pick_order,overall_pick_number,team_pick_number,pick_points,roster_points,playoff_team
0,00012b23-0f2c-4727-b1ab-05b4a0120a76,2022-08-19 23:49:44.511157,14400,8d6dd37f-6a5d-420b-bc9c-b14f77588341,c2fd14f4-cfb8-405e-821d-6f577ce5be45,69c11d7e-b0c4-4771-b4a1-2e0cdce43da2,1,DJ Moore,WR,13,33.01,7,31,3,104.5,1478.34,0
1,00012b23-0f2c-4727-b1ab-05b4a0120a76,2022-08-19 23:49:44.511157,14400,372c3de1-5233-492f-ab1d-d1455630b9b5,b8f9e652-2362-4df9-b233-acee84698c9f,497c2ba9-4e49-47e3-b0c4-da045e76c2f7,1,James Cook,RB,7,102.69,9,105,9,35.5,1720.66,0
2,00012b23-0f2c-4727-b1ab-05b4a0120a76,2022-08-19 23:49:44.511157,14400,372c3de1-5233-492f-ab1d-d1455630b9b5,b8f9e652-2362-4df9-b233-acee84698c9f,497c2ba9-4e49-47e3-b0c4-da045e76c2f7,1,Malik Willis,QB,6,214.44,9,208,18,0.0,1720.66,0
3,00012b23-0f2c-4727-b1ab-05b4a0120a76,2022-08-19 23:49:44.511157,14400,a7c15a5c-8925-422c-979b-7c60fd0326c5,364bfa02-93db-417b-8c46-793759ef06d2,abd6356b-d301-4498-828f-f36c6e42af8c,1,Michael Carter,RB,10,140.28,2,146,13,74.1,1579.04,0
4,00012b23-0f2c-4727-b1ab-05b4a0120a76,2022-08-19 23:49:44.511157,14400,8d6dd37f-6a5d-420b-bc9c-b14f77588341,c2fd14f4-cfb8-405e-821d-6f577ce5be45,69c11d7e-b0c4-4771-b4a1-2e0cdce43da2,1,Darren Waller,TE,6,50.96,7,42,4,23.9,1478.34,0


In [11]:
bbm3_cond = bbm3_data.copy()[['draft_time', 'player_name', 'position_name', 'projection_adp']]
bbm3_cond['draft_time'] = pd.to_datetime(bbm3_cond['draft_time']) + pd.Timedelta(hours=-15)
bbm3_cond['draft_time'] = bbm3_cond['draft_time'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
bbm3_cond['draft_date'] = bbm3_cond['draft_time'].str[:10]
bbm3_cond.head(5)

,draft_time,player_name,position_name,projection_adp,draft_date
0,2022-08-19T08:49:44Z,DJ Moore,WR,33.01,2022-08-19
1,2022-08-19T08:49:44Z,James Cook,RB,102.69,2022-08-19
2,2022-08-19T08:49:44Z,Malik Willis,QB,214.44,2022-08-19
3,2022-08-19T08:49:44Z,Michael Carter,RB,140.28,2022-08-19
4,2022-08-19T08:49:44Z,Darren Waller,TE,50.96,2022-08-19


In [27]:
bbm3_date = bbm3_cond[bbm3_cond['draft_date'] == '2022-07-17']
for player_name in bbm3_date['player_name'].unique():
    player_data = bbm3_date[bbm3_date['player_name'] == player_name]
    adp_counts = player_data['projection_adp'].value_counts()
    print(f"\n{player_name}:")
    for adp, count in adp_counts.items():
        print(f"  ADP {adp}: {count} times")


Trey Lance:
  ADP 78.75: 297 times
  ADP 80.21: 10 times
  ADP 80.13: 10 times
  ADP 80.0: 5 times
  ADP 79.32: 3 times
  ADP 80.54: 3 times
  ADP 79.37: 2 times
  ADP 78.94: 1 times
  ADP 80.03: 1 times
  ADP 79.58: 1 times

Jerry Jeudy:
  ADP 47.53: 300 times
  ADP 47.77: 15 times
  ADP 48.04: 8 times
  ADP 47.65: 7 times
  ADP 48.1: 2 times
  ADP 47.62: 1 times

Kareem Hunt:
  ADP 95.46: 297 times
  ADP 94.08: 11 times
  ADP 95.13: 6 times
  ADP 94.94: 6 times
  ADP 94.48: 4 times
  ADP 94.01: 3 times
  ADP 93.8: 3 times
  ADP 94.13: 2 times
  ADP 94.06: 1 times

Jarvis Landry:
  ADP 130.82: 297 times
  ADP 132.81: 7 times
  ADP 132.73: 7 times
  ADP 133.32: 5 times
  ADP 134.97: 4 times
  ADP 133.31: 3 times
  ADP 134.34: 3 times
  ADP 133.73: 2 times
  ADP 133.36: 2 times
  ADP 135.49: 2 times
  ADP 138.97: 1 times

Hunter Henry:
  ADP 149.12: 297 times
  ADP 145.39: 9 times
  ADP 145.68: 8 times
  ADP 145.53: 6 times
  ADP 145.58: 4 times
  ADP 147.36: 2 times
  ADP 145.72: 2 ti

In [20]:
dates = sorted(bbm3_cond['draft_date'].unique())
player_names = bbm3_cond['player_name'].unique()

col_names = ['player_id', 'player_name', 'position'] + dates
bbm3_adp = pd.DataFrame(columns=col_names)

player_info = bbm3_cond.groupby('player_name').first()[['position_name']].reset_index()

for player_name in player_names:
    player_data = player_info[player_info['player_name'] == player_name].iloc[0]
    row_data = [None, player_data['player_name'], player_data['position_name']] + [None] * len(dates)
    bbm3_adp.loc[len(bbm3_adp)] = row_data

In [21]:
for date in dates:
    bbm3_on_date = bbm3_cond[bbm3_cond['draft_date'] == date]
    for idx, row in bbm3_adp.iterrows():
        player_data = bbm3_on_date[bbm3_on_date['player_name'] == row['player_name']]
        adp_counts = player_data['projection_adp'].value_counts()
        if len(adp_counts) == 0:
            adp = 216.0
        else:
            adp = adp_counts.index[0]
        bbm3_adp.loc[idx, date] = adp

In [22]:
bbm3_adp.to_csv(r'C:\Users\Jona\Desktop\tuxff\data\bbm3_adp.csv')
bbm3_adp.head(5)

,player_id,player_name,position,2022-05-02,2022-05-03,2022-05-04,2022-05-05,2022-05-06,2022-05-07,2022-05-08,...,2022-08-29,2022-08-30,2022-08-31,2022-09-01,2022-09-02,2022-09-03,2022-09-04,2022-09-05,2022-09-06,2022-09-07
0,None,DJ Moore,WR,41.0,41.65,41.32,40.47,39.52,38.76,37.95,...,32.88,32.8,32.77,32.83,32.89,32.89,33.02,33.15,32.97,32.9
1,None,James Cook,RB,0.0,146.94,138.06,124.35,119.54,115.82,113.22,...,103.45,104.39,104.54,104.35,104.91,104.95,104.86,105.42,106.67,107.34
2,None,Malik Willis,QB,164.0,197.84,203.35,212.01,214.59,215.15,215.32,...,214.82,214.44,214.44,214.32,214.3,214.35,214.28,214.16,214.25,214.34
3,None,Michael Carter,RB,67.0,91.04,97.63,108.53,115.7,120.52,123.76,...,139.32,137.95,137.13,135.54,134.37,133.38,131.99,130.53,129.85,129.48
4,None,Darren Waller,TE,36.0,38.66,38.71,39.24,39.9,40.16,39.86,...,55.34,56.13,56.12,55.3,54.77,54.68,55.36,55.63,55.5,55.56


### ADP Data -> Postgres DB

In [7]:
bbm3_adp = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\adp_data\bbm3_adp.csv', delimiter=',', index_col=0)
bbm3_adp.head(5)

,player_id,player_name,position,2022-05-02,2022-05-03,2022-05-04,2022-05-05,2022-05-06,2022-05-07,2022-05-08,...,2022-08-29,2022-08-30,2022-08-31,2022-09-01,2022-09-02,2022-09-03,2022-09-04,2022-09-05,2022-09-06,2022-09-07
0,6d1e2e6b-00ea-4889-9db1-f6ff25eeb3f1,DJ Moore,WR,41.0,41.65,41.32,40.47,39.52,38.76,37.95,...,32.88,32.80,32.77,32.83,32.89,32.89,33.02,33.15,32.97,32.90
1,7b434d3b-e0f9-40f1-bedd-51f973c59c5f,James Cook,RB,0.0,146.94,138.06,124.35,119.54,115.82,113.22,...,103.45,104.39,104.54,104.35,104.91,104.95,104.86,105.42,106.67,107.34
2,83f5497d-fa47-40e5-bb1d-95b79469d112,Malik Willis,QB,164.0,197.84,203.35,212.01,214.59,215.15,215.32,...,214.82,214.44,214.44,214.32,214.30,214.35,214.28,214.16,214.25,214.34
3,82682a2a-6ca8-411c-b723-2aeaaed8787e,Michael Carter,RB,67.0,91.04,97.63,108.53,115.70,120.52,123.76,...,139.32,137.95,137.13,135.54,134.37,133.38,131.99,130.53,129.85,129.48
4,a843724e-65ce-429f-8ca8-0c3107a64c47,Darren Waller,TE,36.0,38.66,38.71,39.24,39.90,40.16,39.86,...,55.34,56.13,56.12,55.30,54.77,54.68,55.36,55.63,55.50,55.56


In [8]:
bbm3_adp['player_id'] = pd.Series(dtype='object')
for idx, row in bbm3_adp.iterrows():
    cursor.execute("""SELECT player_id 
                   FROM player_info
                   WHERE player_name = %s""",
                   (row['player_name'],))
    
    result = cursor.fetchone()
    
    if result:
        bbm3_adp.loc[idx, 'player_id'] = result['player_id']
    
    else:
        bbm3_adp.loc[idx, 'player_id'] = str(uuid.uuid4())

KeyboardInterrupt: 

In [15]:
bbm3_adp.to_csv(r'C:\Users\Jona\Desktop\tuxff\data\adp_data\bbm3_adp.csv')
bbm3_adp.head(5)

,player_id,player_name,position,2022-05-02,2022-05-03,2022-05-04,2022-05-05,2022-05-06,2022-05-07,2022-05-08,...,2022-08-29,2022-08-30,2022-08-31,2022-09-01,2022-09-02,2022-09-03,2022-09-04,2022-09-05,2022-09-06,2022-09-07
0,b618ab1d-ad59-4cdb-8687-3bdb35fd8cc0,DJ Moore,WR,41.0,41.65,41.32,40.47,39.52,38.76,37.95,...,32.88,32.80,32.77,32.83,32.89,32.89,33.02,33.15,32.97,32.90
1,7b434d3b-e0f9-40f1-bedd-51f973c59c5f,James Cook,RB,0.0,146.94,138.06,124.35,119.54,115.82,113.22,...,103.45,104.39,104.54,104.35,104.91,104.95,104.86,105.42,106.67,107.34
2,83f5497d-fa47-40e5-bb1d-95b79469d112,Malik Willis,QB,164.0,197.84,203.35,212.01,214.59,215.15,215.32,...,214.82,214.44,214.44,214.32,214.30,214.35,214.28,214.16,214.25,214.34
3,82682a2a-6ca8-411c-b723-2aeaaed8787e,Michael Carter,RB,67.0,91.04,97.63,108.53,115.70,120.52,123.76,...,139.32,137.95,137.13,135.54,134.37,133.38,131.99,130.53,129.85,129.48
4,a843724e-65ce-429f-8ca8-0c3107a64c47,Darren Waller,TE,36.0,38.66,38.71,39.24,39.90,40.16,39.86,...,55.34,56.13,56.12,55.30,54.77,54.68,55.36,55.63,55.50,55.56


In [8]:
for idx, row in bbm3_adp.iterrows():
    cursor.execute("""
        INSERT INTO player_info (player_id, player_name, position) 
        VALUES (%s, %s, %s)
        ON CONFLICT (player_id) DO NOTHING
    """, (row['player_id'], row['player_name'], row['position']))

conn.commit()

In [11]:
cursor.execute("SELECT player_id, date FROM player_adp")
existing_data = set()
for row in cursor.fetchall():
    date_str = str(row['date'])
    existing_data.add((row['player_id'], date_str))
print('Step 1 Complete')

adp_data = []
for idx, row in bbm3_adp.iterrows():
    for date in bbm3_adp.columns[4:]:
        if (row['player_id'], date) not in existing_data:
            adp_data.append((row['player_id'], date, row[date]))
print(f"Step 2 Complete - {len(adp_data)} new records to insert")

batch_size = 500
for i in range(0, len(adp_data), batch_size):
    batch = adp_data[i:i+batch_size]
    print(f"Starting batch {i//batch_size + 1} with {len(batch)} records...")
    cursor.executemany("""
        INSERT INTO player_adp (player_id, date, adp) 
        VALUES (%s, %s, %s)
    """, batch)
    print(f"Completed batch {i//batch_size + 1}")
    
    conn.commit()
    print(f"Committed batch {i//batch_size + 1}")

Step 1 Complete
Step 2 Complete - 34436 new records to insert
Starting batch 1 with 500 records...
Completed batch 1
Committed batch 1
Starting batch 2 with 500 records...
Completed batch 2
Committed batch 2
Starting batch 3 with 500 records...
Completed batch 3
Committed batch 3
Starting batch 4 with 500 records...
Completed batch 4
Committed batch 4
Starting batch 5 with 500 records...
Completed batch 5
Committed batch 5
Starting batch 6 with 500 records...
Completed batch 6
Committed batch 6
Starting batch 7 with 500 records...
Completed batch 7
Committed batch 7
Starting batch 8 with 500 records...
Completed batch 8
Committed batch 8
Starting batch 9 with 500 records...
Completed batch 9
Committed batch 9
Starting batch 10 with 500 records...
Completed batch 10
Committed batch 10
Starting batch 11 with 500 records...
Completed batch 11
Committed batch 11
Starting batch 12 with 500 records...
Completed batch 12
Committed batch 12
Starting batch 13 with 500 records...
Completed batch

In [ ]:
conn.close()
cursor.close()

# BBM4 (2023)

### Raw Data -> ADP Data

In [3]:
# very large file, takes a long time to load
bbm4_data = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\best_ball_mania_iv_2023_r1_results_pick_by_pick.csv', delimiter=',')

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
print(bbm4_data.columns)
bbm4_data.head(5)

Index(['draft_id', 'user_id', 'username', 'draft_created_time',
       'draft_filled_time', 'draft_time', 'draft_completed_time', 'clock',
       'draft_entry_id', 'tournament_entry_id',
       'tournament_round_draft_entry_id', 'tournament_round_number',
       'player_name', 'player_id', 'position_name', 'projection_adp', 'source',
       'pick_order', 'overall_pick_number', 'team_pick_number',
       'pick_created_time', 'pick_points', 'roster_points', 'made_playoffs'],
      dtype='object')


,draft_id,user_id,username,draft_created_time,draft_filled_time,draft_time,draft_completed_time,clock,draft_entry_id,tournament_entry_id,...,position_name,projection_adp,source,pick_order,overall_pick_number,team_pick_number,pick_created_time,pick_points,roster_points,made_playoffs
0,65219f4a-267e-4035-806a-646a3c1a846e,1a32930d-dbc3-412b-b4d5-8776feb34790,nohalfsteppin,2023-04-25T03:41:37Z,2023-09-03T18:54:09Z,2023-09-03T18:55:15Z,2023-09-03T19:42:23Z,30,a07d4470-1381-4151-b439-c13b67fc9e1c,375e1161-0892-4b8d-a00f-3a5936f4b23c,...,WR,8.57,user,6,6,1,2023-09-03T18:56:23Z,171.10,2091.34,0
1,65219f4a-267e-4035-806a-646a3c1a846e,1a32930d-dbc3-412b-b4d5-8776feb34790,nohalfsteppin,2023-04-25T03:41:37Z,2023-09-03T18:54:09Z,2023-09-03T18:55:15Z,2023-09-03T19:42:23Z,30,a07d4470-1381-4151-b439-c13b67fc9e1c,375e1161-0892-4b8d-a00f-3a5936f4b23c,...,QB,27.26,user,6,19,2,2023-09-03T18:58:43Z,247.84,2091.34,0
2,65219f4a-267e-4035-806a-646a3c1a846e,1a32930d-dbc3-412b-b4d5-8776feb34790,nohalfsteppin,2023-04-25T03:41:37Z,2023-09-03T18:54:09Z,2023-09-03T18:55:15Z,2023-09-03T19:42:23Z,30,a07d4470-1381-4151-b439-c13b67fc9e1c,375e1161-0892-4b8d-a00f-3a5936f4b23c,...,WR,34.95,user,6,30,3,2023-09-03T19:00:59Z,204.86,2091.34,0
3,65219f4a-267e-4035-806a-646a3c1a846e,1a32930d-dbc3-412b-b4d5-8776feb34790,nohalfsteppin,2023-04-25T03:41:37Z,2023-09-03T18:54:09Z,2023-09-03T18:55:15Z,2023-09-03T19:42:23Z,30,a07d4470-1381-4151-b439-c13b67fc9e1c,375e1161-0892-4b8d-a00f-3a5936f4b23c,...,WR,39.76,user,6,43,4,2023-09-03T19:02:58Z,52.10,2091.34,0
4,65219f4a-267e-4035-806a-646a3c1a846e,1a32930d-dbc3-412b-b4d5-8776feb34790,nohalfsteppin,2023-04-25T03:41:37Z,2023-09-03T18:54:09Z,2023-09-03T18:55:15Z,2023-09-03T19:42:23Z,30,a07d4470-1381-4151-b439-c13b67fc9e1c,375e1161-0892-4b8d-a00f-3a5936f4b23c,...,TE,53.16,user,6,54,5,2023-09-03T19:05:16Z,146.40,2091.34,0


In [37]:
bbm4_cond = bbm4_data.copy()[['player_name', 'player_id', 'position_name', 'projection_adp', 'pick_created_time']]
bbm4_cond['pick_created_time'] = pd.to_datetime(bbm4_cond['pick_created_time']) + pd.Timedelta(hours=-15)
bbm4_cond['pick_created_time'] = bbm4_cond['pick_created_time'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
bbm4_cond.head(5)

,player_name,player_id,position_name,projection_adp,pick_created_time
0,Stefon Diggs,25ea41e7-0529-48c6-ba65-7648bb7f9ed2,WR,8.57,2023-09-03T03:56:23Z
1,Josh Allen,99398db6-2ffb-42e9-bdb0-4312a399ab08,QB,27.26,2023-09-03T03:58:43Z
2,Keenan Allen,d74acd73-504a-4c9b-8aa6-fe602de560a8,WR,34.95,2023-09-03T04:00:59Z
3,Christian Watson,c108c752-92fd-491f-94ed-60b0b3cea7d8,WR,39.76,2023-09-03T04:02:58Z
4,T.J. Hockenson,660a9e72-4fc7-442d-ba62-257dbab39568,TE,53.16,2023-09-03T04:05:16Z


In [38]:
bbm4_cond_date = bbm4_cond.copy()
bbm4_cond_date['pick_created_time'] = bbm4_cond_date['pick_created_time'].str[:10]
bbm4_cond_date.head(5)

,player_name,player_id,position_name,projection_adp,pick_created_time
0,Stefon Diggs,25ea41e7-0529-48c6-ba65-7648bb7f9ed2,WR,8.57,2023-09-03
1,Josh Allen,99398db6-2ffb-42e9-bdb0-4312a399ab08,QB,27.26,2023-09-03
2,Keenan Allen,d74acd73-504a-4c9b-8aa6-fe602de560a8,WR,34.95,2023-09-03
3,Christian Watson,c108c752-92fd-491f-94ed-60b0b3cea7d8,WR,39.76,2023-09-03
4,T.J. Hockenson,660a9e72-4fc7-442d-ba62-257dbab39568,TE,53.16,2023-09-03


In [39]:
bbm4_05_10 = bbm4_cond_date[bbm4_cond_date['pick_created_time'] == '2023-08-14']
for player_name in bbm4_05_10['player_name'].unique():
    player_data = bbm4_05_10[bbm4_05_10['player_name'] == player_name]
    adp_counts = player_data['projection_adp'].value_counts()
    print(f"\n{player_name}:")
    for adp, count in adp_counts.items():
        print(f"  ADP {adp}: {count} times")


T.J. Hockenson:
  ADP 52.27: 657 times
  ADP 52.42: 2 times

Marquise Brown:
  ADP 61.87: 674 times
  ADP 61.53: 4 times

Sam LaPorta:
  ADP 150.11: 653 times
  ADP 150.0: 2 times

Raheem Mostert:
  ADP 158.07: 682 times
  ADP 158.67: 3 times

Tyreek Hill:
  ADP 4.13: 666 times
  ADP 4.1: 2 times

Derrick Henry:
  ADP 21.03: 669 times

Josh Allen:
  ADP 27.49: 667 times
  ADP 27.44: 3 times

Mike Williams:
  ADP 44.61: 643 times
  ADP 44.51: 2 times

Kenneth Walker III:
  ADP 53.6: 664 times
  ADP 53.59: 3 times

Michael Pittman Jr.:
  ADP 71.03: 654 times
  ADP 70.15: 2 times

George Kittle:
  ADP 62.62: 674 times
  ADP 62.64: 2 times

Elijah Moore:
  ADP 87.49: 659 times
  ADP 87.3: 2 times

AJ Dillon:
  ADP 101.18: 659 times
  ADP 101.38: 2 times

David Njoku:
  ADP 106.7: 652 times
  ADP 106.56: 2 times

Jakobi Meyers:
  ADP 126.91: 639 times
  ADP 126.73: 2 times

Adam Thielen:
  ADP 148.7: 635 times
  ADP 148.85: 3 times

Marvin Mims Jr.:
  ADP 145.02: 655 times
  ADP 144.48: 5 

In [42]:
dates = sorted(bbm4_cond_date['pick_created_time'].unique())
player_ids = bbm4_cond_date['player_id'].unique()

col_names = ['player_id', 'player_name', 'position'] + dates
bbm4_adp = pd.DataFrame(columns=col_names)

player_info = bbm4_cond_date.groupby('player_id').first()[['player_name', 'position_name']].reset_index()

for player_id in player_ids:
    player_data = player_info[player_info['player_id'] == player_id].iloc[0]
    row_data = [player_id, player_data['player_name'], player_data['position_name']] + [None] * len(dates)
    bbm4_adp.loc[len(bbm4_adp)] = row_data

In [46]:
for date in dates:
    bbm4_date = bbm4_cond_date[bbm4_cond_date['pick_created_time'] == date]
    for idx, row in bbm4_adp.iterrows():
        player_data = bbm4_date[bbm4_date['player_id'] == row['player_id']]
        adp_counts = player_data['projection_adp'].value_counts()
        if len(adp_counts) == 0:
            adp = 216.0
        else:
            adp = adp_counts.index[0]
        bbm4_adp.loc[idx, date] = adp

In [6]:
bbm4_adp.to_csv(r'C:\Users\Jona\Desktop\tuxff\data\bbm4_adp.csv')
bbm4_adp.head(5)

NameError: name 'bbm4_adp' is not defined

### ADP Data -> Postgres DB

In [4]:
bbm4_adp = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\adp_data\bbm4_adp.csv', delimiter=',')
bbm4_adp.head(5)

,Unnamed: 0,player_id,player_name,position,2023-04-29,2023-04-30,2023-05-01,2023-05-02,2023-05-03,2023-05-04,...,2023-08-28,2023-08-29,2023-08-30,2023-08-31,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06
0,0,25ea41e7-0529-48c6-ba65-7648bb7f9ed2,Stefon Diggs,WR,8.0,8.64,8.25,8.00,7.91,7.99,...,8.87,8.84,8.74,8.77,8.72,8.61,8.57,8.56,8.29,7.83
1,1,99398db6-2ffb-42e9-bdb0-4312a399ab08,Josh Allen,QB,14.0,15.51,15.85,16.55,17.23,17.61,...,26.73,26.86,27.13,27.36,27.43,27.42,27.26,27.03,26.92,26.64
2,2,d74acd73-504a-4c9b-8aa6-fe602de560a8,Keenan Allen,WR,52.0,52.71,52.89,53.00,52.87,52.47,...,35.73,35.76,35.41,35.09,35.00,34.97,34.95,35.00,35.06,35.01
3,3,c108c752-92fd-491f-94ed-60b0b3cea7d8,Christian Watson,WR,48.0,48.17,47.88,47.17,46.13,45.85,...,40.35,40.26,40.03,39.70,39.60,39.70,39.76,39.72,39.39,39.18
4,4,660a9e72-4fc7-442d-ba62-257dbab39568,T.J. Hockenson,TE,37.0,39.81,40.84,41.93,42.99,43.84,...,54.02,53.96,53.75,53.82,53.32,52.98,53.16,52.88,52.04,51.45


In [28]:
for idx, row in bbm4_adp.iterrows():
    cursor.execute("""
        INSERT INTO player_info (player_id, player_name, position) 
        VALUES (%s, %s, %s)
        ON CONFLICT (player_id) DO NOTHING
    """, (row['player_id'], row['player_name'], row['position']))

conn.commit()

In [26]:
cursor.execute("SELECT player_id, date FROM player_adp")
existing_data = set()
for row in cursor.fetchall():
    existing_data.add((row['player_id'], row['date']))
print('Step 1 Complete')

adp_data = []
for idx, row in bbm4_adp.iterrows():
    for date in bbm4_adp.columns[4:]:
        if (row['player_id'], date) not in existing_data:
            adp_data.append((row['player_id'], date, row[date]))
print(f"Step 2 Complete - {len(adp_data)} new records to insert")

batch_size = 500
for i in range(0, len(adp_data), batch_size):
    batch = adp_data[i:i+batch_size]
    print(f"Starting batch {i//batch_size + 1} with {len(batch)} records...")
    cursor.executemany("""
        INSERT INTO player_adp (player_id, date, adp) 
        VALUES (%s, %s, %s)
    """, batch)
    print(f"Completed batch {i//batch_size + 1}")
    
    conn.commit()
    print(f"Committed batch {i//batch_size + 1}")

Step 1 Complete
Step 2 Complete - 96940 new records to insert
Starting batch 1 with 500 records...
Completed batch 1
Committed batch 1
Starting batch 2 with 500 records...
Completed batch 2
Committed batch 2
Starting batch 3 with 500 records...
Completed batch 3
Committed batch 3
Starting batch 4 with 500 records...
Completed batch 4
Committed batch 4
Starting batch 5 with 500 records...
Completed batch 5
Committed batch 5
Starting batch 6 with 500 records...
Completed batch 6
Committed batch 6
Starting batch 7 with 500 records...
Completed batch 7
Committed batch 7
Starting batch 8 with 500 records...
Completed batch 8
Committed batch 8
Starting batch 9 with 500 records...
Completed batch 9
Committed batch 9
Starting batch 10 with 500 records...
Completed batch 10
Committed batch 10
Starting batch 11 with 500 records...
Completed batch 11
Committed batch 11
Starting batch 12 with 500 records...
Completed batch 12
Committed batch 12
Starting batch 13 with 500 records...
Completed batch

In [27]:
conn.close()
cursor.close()

# BBM5 (2024)

In [22]:
bbm5_adp = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\adp_data\bbm5_adp.csv', delimiter=',')
bbm5_adp.head(5)

,Unnamed: 0,player_id,player_name,position,2024-04-29,2024-04-30,2024-05-01,2024-05-02,2024-05-03,2024-05-04,...,2024-08-22,2024-08-23,2024-08-24,2024-08-25,2024-08-26,2024-08-27,2024-08-28,2024-08-29,2024-08-30,2024-08-31
0,0,cb7b1dc6-91ce-47b8-813c-07486371a922,Christian McCaffrey,RB,1.0,1.11,1.15,1.18,1.20,1.22,...,1.23,1.22,1.21,1.21,1.23,1.27,1.30,1.30,1.29,1.27
1,1,4de8084d-aa1a-43c7-9676-0def5b39e980,CeeDee Lamb,WR,2.0,2.26,2.25,2.19,2.14,2.13,...,2.31,2.33,2.37,2.38,2.38,2.19,2.06,2.05,2.08,2.10
2,2,84cf37a2-d9f9-4315-a56a-c003a186023d,Tyreek Hill,WR,3.0,3.15,3.26,3.25,3.22,3.23,...,2.87,2.86,2.85,2.83,2.82,2.89,2.96,2.96,2.95,2.96
3,3,19818cc6-e21b-4258-9315-93cb64fb92e7,Ja'Marr Chase,WR,4.0,4.08,4.08,4.07,4.07,4.07,...,4.80,4.91,4.98,4.99,4.90,4.85,4.82,4.94,5.23,5.36
4,4,d61fb310-35c9-4f90-a99c-fb4605a870f4,Justin Jefferson,WR,6.0,5.82,5.65,5.47,5.32,5.26,...,6.27,6.28,6.27,6.26,6.32,6.50,6.55,6.44,6.23,6.27


In [18]:
for idx, row in bbm5_adp.iterrows():
    cursor.execute("""
        INSERT INTO player_info (player_id, player_name, position) 
        VALUES (%s, %s, %s)
        ON CONFLICT (player_id) DO UPDATE SET
            player_name = EXCLUDED.player_name,
            position = EXCLUDED.position
    """, (row['player_id'], row['player_name'], row['position']))

conn.commit()

In [ ]:
for idx, row in bbm5_adp.iterrows():
    for date in bbm5_adp.columns[4:]:
        cursor.execute("""
            INSERT INTO player_adp (player_id, date, adp) 
            VALUES (%s, %s, %s)
            ON CONFLICT (player_id) DO UPDATE SET
                player_name = EXCLUDED.player_name,
                position = EXCLUDED.position
        """, (row['player_id'], date, row[date]))

conn.commit()

# Add BBM6 players to postgres db